In [2]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

2.0.1


In [3]:
import os.path as osp
import torch
from torchmetrics import MeanSquaredLogError
from torch_geometric.loader import DataLoader
import ml_pc_saft
from graphdataset import ThermoMLDataset, get_padded_array
import jax

x64 on


In [5]:
torch.cuda.is_available(), jax.local_devices()[0].platform

(True, 'gpu')

In [5]:

path = osp.join("data", "thermoml")
train_dataset = ThermoMLDataset(path, subset="train")
val_dataset = ThermoMLDataset(path, subset="val")
test_dataset = ThermoMLDataset(path, subset="test")

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
pcsaft_layer = ml_pc_saft.PCSAFT_layer.apply
pcsaft_layer_test = ml_pc_saft.PCSAFT_layer_test.apply
lossfn = MeanSquaredLogError().to(device)

In [33]:
parameters = (torch.square(torch.rand(9)) + 0.000001).to(device, torch.float64).requires_grad_()
print(parameters.is_leaf)
optimizer = torch.optim.AdamW(
            [parameters],
            lr=1.0,
            weight_decay=0,
            amsgrad=True,
        )
for graphs in train_loader:
    graphs = graphs.to(device)
    for _ in range(200):
        datapoints = graphs.states.view(-1, 5)
        datapoints = get_padded_array(datapoints, 64)
        datapoints = datapoints.to(device)
        optimizer.zero_grad()
        parameters=parameters.squeeze()
        pred_y = pcsaft_layer(parameters, datapoints)
        y = datapoints[:, -1]
        loss = lossfn(pred_y, y)
        loss.backward()
        optimizer.step()
        print(parameters)
        print(loss)

True
tensor([1.3992e-08, 1.0000e+00, 1.4900e+02, 1.0000e+00, 1.0003e+03, 2.7169e-05,
        2.5000e+00], device='cuda:0', dtype=torch.float64,
       grad_fn=<AsStridedBackward0>)
tensor(3.5207, device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([ 7.4414e-01,  1.7390e+00,  1.4825e+02,  1.6697e+00,  1.0005e+03,
        -7.4411e-01,  1.9151e+00], device='cuda:0', dtype=torch.float64,
       grad_fn=<AsStridedBackward0>)
tensor(571.6595, device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([ 1.3191e+00,  2.3725e+00,  1.4771e+02,  2.1804e+00,  9.9983e+02,
        -1.2557e-01,  1.2763e+00], device='cuda:0', dtype=torch.float64,
       grad_fn=<AsStridedBackward0>)
tensor(17.7095, device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([1.7898e+00, 2.9136e+00, 1.4721e+02, 2.4438e+00, 9.9924e+02, 4.0451e-01,
        7.4727e-01], device='cuda:0', dtype=torch.float64,
       grad_fn=<AsStridedBackward0>)
tensor(4.9651, device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([2.1874e+00, 3.3809e+00, 1.

KeyboardInterrupt: 